In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Añadir el directorio 'src' al path para poder importar los módulos
# Asumiendo que el notebook está en la carpeta 'notebooks/' o 'exploracion/'
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Importar tus módulos
import load_data
import train_model
import explicabilidad  # Este es tu nuevo fichero

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Ruta a tu dataset
# Ajusta esto según dónde tengas tu CSV, por ejemplo: '../data/creditcard.csv'
csv_path = '../data/credit_card.csv' 

try:
    print("Cargando datos...")
    # Usamos tu función existente para cargar y separar X e y
    df, X, y = load_data.load_fraud_csv(csv_path)
    
    # Separamos el 'Amount' para el entrenamiento con costes (como haces en train_model.py)
    amount_train = X['Amount'] 
    
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {csv_path}. Verifica la ruta.")

Cargando datos...
Loaded 284,807 transactions from credit_card.csv
Fraud rate: 0.17275%
Total fraud amount: $60,127.97
Datos cargados correctamente.


In [ ]:
# División estratificada (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Recuperamos la columna 'Amount' correspondiente a los índices de train
amount_train_split = X.loc[X_train.index, 'Amount']

print(f"Entrenamiento: {X_train.shape}, Test: {X_test.shape}")

Entrenamiento: (199364, 29), Test: (85443, 29)


In [ ]:
print("Entrenando modelo XGBoost con coste sensible...")

model = train_model.train_xgb_with_cost(
    X_train, 
    y_train, 
    amount_train_split, 
    amount_factor=20
)

print("Modelo entrenado.")

Entrenando modelo XGBoost con coste sensible...
XGBoost trained (factor=20)
Modelo entrenado.


In [ ]:
print("Inicializando explicador SHAP...")
# Llamamos a la función de tu nuevo archivo
explainer = explicabilidad.iniciar_explainer(model, X_train)
print("Explicador listo.")

Inicializando explicador SHAP...


InvalidModelError: Model type not yet supported by TreeExplainer: <class 'str'>

In [ ]:
print("Generando explicación global del modelo...")
# Usamos una muestra del test para agilizar el cálculo si el dataset es muy grande
X_test_sample = X_test.sample(1000, random_state=42) if len(X_test) > 1000 else X_test

explicabilidad.explicar_global(explainer, X_test_sample)

Generando explicación global del modelo...


NameError: name 'explainer' is not defined

In [ ]:
# 1. Buscamos un caso real de fraude en el set de prueba para explicarlo
# Filtramos los índices donde la etiqueta real es 1 (Fraude)
indices_fraude = y_test[y_test == 1].index

if len(indices_fraude) > 0:
    # Tomamos el primer fraude encontrado
    id_fraude_real = indices_fraude[0]
    
    # Necesitamos la posición numérica (iloc) de este índice en X_test para pasárselo a tu función
    posicion_en_test = X_test.index.get_loc(id_fraude_real)
    
    print(f"Analizando transacción fraudulenta (Índice original: {id_fraude_real})")
    
    # Llamamos a tu función de explicación individual
    explicabilidad.explicar_prediccion_individual(explainer, X_test, posicion_en_test)
else:
    print("No se encontraron casos de fraude en el set de prueba para analizar.")